In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("sehir1.csv",encoding='ISO-8859-1',delimiter=";")

df

,Plaka,Sehir
0,1,Adana
1,2,Adiyaman
2,3,Afyon
3,4,Agri
4,5,Amasya
...,...,...
76,77,Yalova
77,78,Karabuk
78,79,Kilis
79,80,Osmaniye


In [3]:
df_matrix = pd.DataFrame(np.random.randint(0,100,size=(81, 81)), columns=df['Sehir'].unique(), index=df['Sehir'].unique())

df_matrix

,Adana,Adiyaman,Afyon,Agri,Amasya,Ankara,Antalya,Artvin,Aydin,Balikesir,...,Batman,Sirnak,Bartin,Ardahan,Igdir,Yalova,Karabuk,Kilis,Osmaniye,Duzce
Adana,56,45,75,75,70,98,84,41,43,52,...,60,36,86,60,94,83,56,48,79,90
Adiyaman,5,44,44,19,66,81,11,43,70,74,...,38,37,85,72,83,86,36,14,14,79
Afyon,29,95,68,11,93,53,30,14,90,1,...,12,68,2,65,55,9,36,87,52,10
Agri,64,65,22,9,96,69,65,49,81,99,...,55,65,72,40,23,62,64,19,41,92
Amasya,44,30,35,38,32,95,24,43,46,4,...,38,68,21,69,41,8,39,28,55,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yalova,49,21,3,98,55,98,21,10,18,51,...,63,87,61,76,29,32,90,41,81,46
Karabuk,44,50,15,24,57,67,55,5,58,36,...,51,12,62,4,21,92,92,54,75,59
Kilis,35,40,64,90,18,45,97,38,65,91,...,21,45,38,36,94,42,68,93,41,56
Osmaniye,55,85,59,19,44,42,52,67,55,27,...,48,47,69,83,69,44,52,74,30,66


In [4]:
df_cols = df_matrix.columns

In [5]:
df2 = df_matrix.stack()

df2 = pd.DataFrame(df2)

df3 = df2.reset_index()

df3.columns = ['Sehir1','Sehir2','Kisi_Sayisi']

df3 = df3[df3.Sehir1 != df3.Sehir2]

df3

,Sehir1,Sehir2,Kisi_Sayisi
1,Adana,Adiyaman,45
2,Adana,Afyon,75
3,Adana,Agri,75
4,Adana,Amasya,70
5,Adana,Ankara,98
...,...,...,...
6555,Duzce,Igdir,67
6556,Duzce,Yalova,69
6557,Duzce,Karabuk,51
6558,Duzce,Kilis,74


In [ ]:
d = {}
for i in df3['Sehir1'].unique():
    d[i] = [[df3['Kisi_Sayisi'][j] for j in df3[df3['Sehir1']==i].index]]
    d[i].append([df3['Sehir2'][j] for j in df3[df3['Sehir1']==i].index])


from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=0)

k_dict = {}
centers={}
for k,v in d.items():
        k_dict[k] = kmeans.fit_predict(pd.DataFrame(d[k][0]))
        centers[k] = kmeans.cluster_centers_

for k,v in k_dict.items():
    d[k].append(k_dict[k].tolist())


dc= pd.DataFrame.from_dict(d,orient='columns').T.reset_index()
dc.columns = ['Sehir1','a','Sehir2','Clus']
dc = dc[['Sehir1','Sehir2','Clus']]

lst_cols = ['Sehir2','Clus']
lst_col1 = 'Sehir2'
lst_col2 = 'Clus'

r= pd.DataFrame({
    col:np.repeat(dc[col].values, dc[lst_col1].str.len())
    for col in dc.columns.drop(lst_cols)}).assign(**{lst_col1:np.concatenate(dc[lst_col1])})

r = r.assign(**{lst_col2:np.concatenate(dc[lst_col2])})


df4 = df3
result= pd.merge(df4,r,on=['Sehir1','Sehir2'], how='inner')

result=result[['Sehir1','Sehir2','Kisi_Sayisi','Clus']]

for i in range(len(result)):
    for k,v in centers.items():
        if (result.loc[i,'Sehir1'] == k) & (result.loc[i,'Clus'] == 0):
            result.loc[i,'Centers'] = centers[k][0][0]
        elif (result.loc[i,'Sehir1'] == k) & (result.loc[i,'Clus'] == 1):
            result.loc[i,'Centers'] = centers[k][1][0]
        elif (result.loc[i,'Sehir1'] == k) & (result.loc[i,'Clus'] == 2):
            result.loc[i,'Centers'] = centers[k][2][0]
            
            
            
for i in range(len(result)):
    for k,v in centers.items():
        if (result.loc[i,'Sehir1'] == k) & (result.loc[i,'Centers'] == centers[k].max()):
            result.loc[i,'Cluster_Interpret'] = 'Biggest Population'
        elif (result.loc[i,'Sehir1'] == k) & (result.loc[i,'Centers'] == centers[k].min()):
            result.loc[i,'Cluster_Interpret'] = 'Smallest Population'
        elif (result.loc[i,'Sehir1'] == k) & (result.loc[i,'Centers'] == np.median(centers[k])):
            result.loc[i,'Cluster_Interpret'] = 'Mid Level Population'
result

# ***

### Hincal Topcuoglu